In [1]:
import numpy as np
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

import matplotlib.pyplot as plt
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [2]:
N = 21; # Number of atomic sites
M = int((N-1)/2); # Number of atomic sites on each side of the origin
a = 1.0 # Real space distance between each site
lattx = np.arange(-a*M,a*M+1,a); # '+1' because np.arange doesn't include the endpoint
latty = np.zeros(lattx.size);
lattice = figure(plot_height=100, plot_width=800, title="1D atomic chain",x_range=[-1.1*a*M,1.1*a*M]);
source1 = ColumnDataSource(data = {'xVal': lattx, 'yVal': latty});
lattice.circle('xVal','yVal',source = source1, size = 10, line_color = 'black', fill_color = 'black');
lattice.xaxis.axis_label = 'Real space distance (arb. unit)';
show(lattice)

In [3]:
k0 = 2*np.pi/(2*M*a);
k = np.linspace(-M*k0,M*k0);
energy = -2*np.cos(k);

In [4]:
# Calcualte resulting dispersion relation using periodic boundary condition (PC)
band = figure(plot_height = 400, plot_width = 500, title='Energy band in reciprocal space', x_range = [-1.1*k0*M/np.pi,1.1*k0*M/np.pi], y_range= [1.2*energy.min(),1.2*energy.max()]);
source2 = ColumnDataSource(data = {'xVal': k/np.pi, 'yVal' : energy});
band.circle('xVal','yVal',source = source2,size = 6);
band.line('xVal','yVal',source = source2);
band.xaxis.axis_label='k-vector ( \u03c0 )';
band.yaxis.axis_label='Energy (arb. Unit)';
show(band)

In [5]:
# define the initial wavefunction (Gaussian dist.)
sigm = a/4;
x0 = 0;
x = np.linspace(-a*M,a*M+1,200);
phi0 = 1/(np.sqrt(2*np.pi)*sigm)*np.exp(-1/2*( (x-x0)/sigm )**2);
rho0 = np.conj(phi0)*phi0;

In [6]:
# Initialize an electron at the center of the 1D chain with a gaussian distribution to indicates its probability density
dist0 = figure(plot_height = 400, plot_width = 700, title='Initial wavefunction in real space', x_range = [-1.1*a*M,1.1*a*M], y_range = [-1, 1.2]);
source3 = ColumnDataSource(data = {'xVal': x, 'yVal': rho0/rho0.max()});
dist0.line('xVal','yVal', source = source3);
dist0.circle('xVal','yVal',source = source1, size = 10, line_color = 'black', fill_color = 'black');
dist0.xaxis.axis_label='Real space distance (arb. unit)';
dist0.yaxis.axis_label='Probability density';
show(dist0)

In [7]:
# Construct hamiltonian using occupation number basis and set initial wavefunction
a = np.zeros((N-1,N-1),float);
np.fill_diagonal(a,-1);
b = np.append(a,np.zeros((N-1,1),float), axis = 1);
c = np.append(np.zeros((1,N),float),b,axis = 0);
hamil = c + np.transpose(c);
psi0 = np.zeros((N,1),float);
psi0[M][0] = 1;

In [22]:
# Define time evolution operator
dt = 0.1; # Define time step
evol = np.exp(-1j*hamil*dt);
ts = np.arange(0,51*dt,dt);
psis = np.empty(ts.size,dtype = object); # Initiate a container for the wavefunctions at different time slice
for ii in range(0,ts.size):
    psis[ii] = np.dot(evol**ts[ii],psi0);
rhos = np.conj(psis)*psis;